In [4]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector

#Connect to db
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="rashad",
    database="attendancedb"
)

#Cursor
cursor = db.cursor()

# Create a table if not exists
cursor.execute("""CREATE TABLE IF NOT EXISTS students (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(255),
                    roll_no VARCHAR(20),
                    attendance_status VARCHAR(10)
                )""")

def mark_attendance(student_id, status):
    cursor.execute("UPDATE students SET attendance_status = %s WHERE id = %s", (status, student_id))
    db.commit()
    messagebox.showinfo("Success", "Attendance marked successfully")
    display_students()

def display_students():
    for i in student_tree.get_children():
        student_tree.delete(i)
    cursor.execute("SELECT * FROM students")
    students = cursor.fetchall()
    for student in students:
        student_tree.insert("", tk.END, values=student)

def add_student():
    name = name_entry.get()
    roll_no = roll_no_entry.get()
    if name and roll_no:
        cursor.execute("INSERT INTO students (name, roll_no, attendance_status) VALUES (%s, %s, %s)", (name, roll_no, "Absent"))
        db.commit()
        messagebox.showinfo("Success", "Student added successfully")
        display_students()
        name_entry.delete(0, tk.END)
        roll_no_entry.delete(0, tk.END)
    else:
        messagebox.showerror("Error", "All fields are required")

def mark_present():
    selected_item = student_tree.selection()
    if selected_item:
        student_id = student_tree.item(selected_item)['values'][0]
        mark_attendance(student_id, "Present")
    else:
        messagebox.showwarning("Warning", "Please select a student from the list.")

def mark_absent():
    selected_item = student_tree.selection()
    if selected_item:
        student_id = student_tree.item(selected_item)['values'][0]
        mark_attendance(student_id, "Absent")
    else:
        messagebox.showwarning("Warning", "Please select a student from the list.")

def search_student():
    query = search_entry.get()
    for i in student_tree.get_children():
        student_tree.delete(i)
    cursor.execute("SELECT * FROM students WHERE name LIKE %s OR roll_no LIKE %s", (f'%{query}%', f'%{query}%'))
    students = cursor.fetchall()
    for student in students:
        student_tree.insert("", tk.END, values=student)

def edit_student():
    selected_item = student_tree.selection()
    if selected_item:
        student_id = student_tree.item(selected_item)['values'][0]
        name = name_entry.get()
        roll_no = roll_no_entry.get()
        if name and roll_no:
            cursor.execute("UPDATE students SET name = %s, roll_no = %s WHERE id = %s", (name, roll_no, student_id))
            db.commit()
            messagebox.showinfo("Success", "Student updated successfully")
            display_students()
        else:
            messagebox.showerror("Error", "All fields are required")
    else:
        messagebox.showwarning("Warning", "Please select a student from the list.")

def delete_student():
    selected_item = student_tree.selection()
    if selected_item:
        student_id = student_tree.item(selected_item)['values'][0]
        cursor.execute("DELETE FROM students WHERE id = %s", (student_id,))
        db.commit()
        messagebox.showinfo("Success", "Student deleted successfully")
        display_students()
    else:
        messagebox.showwarning("Warning", "Please select a student from the list.")

def generate_report():
    cursor.execute("SELECT name, roll_no, attendance_status FROM students")
    students = cursor.fetchall()
    report_window = tk.Toplevel(root)
    report_window.title("Attendance Report")
    report_text = tk.Text(report_window, wrap='word', width=50, height=15)
    report_text.pack(padx=10, pady=10)
    report_text.insert(tk.END, "Name\tRoll No\tAttendance Status\n")
    report_text.insert(tk.END, "-"*40 + "\n")
    for student in students:
        report_text.insert(tk.END, f"{student[0]}\t{student[1]}\t{student[2]}\n")
    report_text.configure(state='disabled')

#main window
root = tk.Tk()
root.title("Student Attendance System")

style = ttk.Style(root)
style.theme_use("clam")

# Create a frame for the treeview
tree_frame = ttk.Frame(root)
tree_frame.pack(pady=10, padx=10)

# Create the treeview
columns = ("ID", "Name", "Roll No", "Attendance Status")
student_tree = ttk.Treeview(tree_frame, columns=columns, show="headings", height=10)
for col in columns:
    student_tree.heading(col, text=col)
    student_tree.column(col, width=120, anchor='center')
student_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# Create a vertical scrollbar for the treeview
tree_scroll = ttk.Scrollbar(tree_frame, orient="vertical", command=student_tree.yview)
tree_scroll.pack(side=tk.RIGHT, fill="y")
student_tree.configure(yscrollcommand=tree_scroll.set)

# Buttons
button_frame = ttk.Frame(root)
button_frame.pack(pady=10)

present_button = ttk.Button(button_frame, text="Mark Present", command=mark_present)
present_button.grid(row=0, column=0, padx=5)

absent_button = ttk.Button(button_frame, text="Mark Absent", command=mark_absent)
absent_button.grid(row=0, column=1, padx=5)

delete_button = ttk.Button(button_frame, text="Delete Student", command=delete_student)
delete_button.grid(row=0, column=2, padx=5)

edit_button = ttk.Button(button_frame, text="Edit Student", command=edit_student)
edit_button.grid(row=0, column=3, padx=5)

report_button = ttk.Button(button_frame, text="Generate Report", command=generate_report)
report_button.grid(row=0, column=4, padx=5)

# Labels and Entry fields for adding/editing students
add_frame = ttk.Frame(root)
add_frame.pack(pady=10)

tk.Label(add_frame, text="Name:").grid(row=0, column=0, padx=5, pady=5)
name_entry = ttk.Entry(add_frame)
name_entry.grid(row=0, column=1, padx=5, pady=5)

tk.Label(add_frame, text="Roll No:").grid(row=1, column=0, padx=5, pady=5)
roll_no_entry = ttk.Entry(add_frame)
roll_no_entry.grid(row=1, column=1, padx=5, pady=5)

add_button = ttk.Button(add_frame, text="Add Student", command=add_student)
add_button.grid(row=2, column=0, columnspan=2, pady=10)

# Search functionality
search_frame = ttk.Frame(root)
search_frame.pack(pady=10)

tk.Label(search_frame, text="Search:").grid(row=0, column=0, padx=5, pady=5)
search_entry = ttk.Entry(search_frame)
search_entry.grid(row=0, column=1, padx=5, pady=5)
search_button = ttk.Button(search_frame, text="Search", command=search_student)
search_button.grid(row=0, column=2, padx=5, pady=5)

# Display initial student list
display_students()

root.mainloop()
